In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")

KSRTC = []

def scrape_page():
    # Locate elements  (container)
    routes = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routes:
        try:
            # Extract route name and link from the route element
            route_name_element = route.find_element(By.XPATH, ".//div[@class='route_details']")
            route_link_element = route.find_element(By.XPATH, ".//a")

            route_name = route_name_element.text
            route_link = route_link_element.get_attribute('href')

            # Append extracted data to list
            KSRTC.append({
                'route_name': route_name,
                'route_link': route_link
            })

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 3):
    scrape_page()
    if page_number < 2:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Print the scraped data
for entry in KSRTC:
    print(entry)

# Convert to DataFrame and save to CSV
df = pd.DataFrame(KSRTC)
df.to_csv("ksrtc_details.csv", index=False)

# Close the driver
driver.quit()

df.head()  # Display the first few rows of the DataFrame

Clicking on page 2
Successfully navigated to page 2
{'route_name': 'Bangalore to Kozhikode\nFrom INR 215', 'route_link': 'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode'}
{'route_name': 'Kozhikode to Ernakulam\nFrom INR 174', 'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam'}
{'route_name': 'Kozhikode to Bangalore\nFrom INR 411', 'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore'}
{'route_name': 'Ernakulam to Kozhikode\nFrom INR 193', 'route_link': 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode'}
{'route_name': 'Kozhikode to Mysore\nFrom INR 147', 'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore'}
{'route_name': 'Kozhikode to Thiruvananthapuram\nFrom INR 215', 'route_link': 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram'}
{'route_name': 'Bangalore to Kalpetta (kerala)\nFrom INR 357', 'route_link': 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta'}
{'route_name': 'Mysore to 

,route_name,route_link
0,Bangalore to Kozhikode\nFrom INR 215,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Ernakulam\nFrom INR 174,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Bangalore\nFrom INR 411,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode\nFrom INR 193,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Kozhikode to Mysore\nFrom INR 147,https://www.redbus.in/bus-tickets/kozhikode-to...
